In [40]:
import pymysql
import pandas as pd

In [112]:
conn = pymysql.connect(
    user = "root",
    host = "localhost",
    password = "mysql12345",
    database = "mysql_dbtest"
)
cursor = conn.cursor()

In [113]:
cursor.execute("DESCRIBE CensusData")
describe = cursor.fetchall()
describe

(('COMMUNITY_AREA_NUMBER', 'varchar(255)', 'YES', '', None, ''),
 ('COMMUNITY_AREA_NAME', 'varchar(255)', 'YES', '', None, ''),
 ('PERCENT_OF_HOUSING_CROWDED', 'varchar(255)', 'YES', '', None, ''),
 ('PERCENT_HOUSEHOLDS_BELOW_POVERTY', 'varchar(255)', 'YES', '', None, ''),
 ('PERCENT_AGED_16__UNEMPLOYED', 'varchar(255)', 'YES', '', None, ''),
 ('PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA',
  'varchar(255)',
  'YES',
  '',
  None,
  ''),
 ('PERCENT_AGED_UNDER_18_OR_OVER_64', 'varchar(255)', 'YES', '', None, ''),
 ('PER_CAPITA_INCOME', 'varchar(255)', 'YES', '', None, ''),
 ('HARDSHIP_INDEX', 'varchar(255)', 'YES', '', None, ''))

1.Find the total number of crimes recorded in the CRIME table.

In [23]:
find_total_c = "SELECT count(*) FROM CrimeData"
cursor.execute(find_total_c)
results = cursor.fetchall()
results[0][0]

533

2.List community areas with per capita income less than 11000.

In [43]:
command = "SELECT * FROM CensusData WHERE PER_CAPITA_INCOME < 11000"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,26,West Garfield Park,9.4,41.7,25.8,24.5,43.6,10934,92
1,30,South Lawndale,15.2,30.7,15.8,54.8,33.8,10402,96
2,37,Fuller Park,3.2,51.2,33.9,26.6,44.9,10432,97
3,54,Riverdale,5.8,56.5,34.6,27.5,51.5,8201,98


3.List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [51]:
command = "SELECT * FROM CrimeData WHERE LOWER(DESCRIPTION) LIKE '%minor%'"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,TRUE,FALSE,...,19,43,7,22,1172680,1916483,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
1,3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,TRUE,FALSE,...,4,8,48,22,1188539,1843379,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


4.List all kidnapping crimes involving a child?

In [52]:
command = "SELECT * FROM CrimeData WHERE PRIMARY_TYPE = 'KIDNAPPING' AND LOWER(DESCRIPTION) LIKE '%child%'"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,...,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


5.What kinds of crimes were recorded at schools

In [55]:
command = "SELECT DISTINCT(PRIMARY_TYPE) FROM CrimeData WHERE LOCATION_DESCRIPTION LIKE '%school%'"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,PRIMARY_TYPE
0,BATTERY
1,CRIMINAL DAMAGE
2,NARCOTICS
3,ASSAULT
4,CRIMINAL TRESPASS
5,PUBLIC PEACE VIOLATION


6.List the average safety score for each type of school

In [115]:
command = "SELECT Elementary_Middle_or_High_School AS Type, AVG(SAFETY_SCORE) FROM PublicSchools GROUP BY Elementary_Middle_or_High_School"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,Type,AVG(SAFETY_SCORE)
0,ES,49.520384
1,HS,49.623529
2,MS,48.000000


7.List 5 community areas with highest % of households below poverty line

In [116]:
command = "SELECT COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CensusData ORDER BY CAST(PERCENT_HOUSEHOLDS_BELOW_POVERTY AS SIGNED) DESC LIMIT 5"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,Washington Park,42.1


8.Which community area is most crime prone?

In [94]:
command_name = "SELECT COMMUNITY_AREA_NAME FROM CensusData WHERE COMMUNITY_AREA_NUMBER = \
(SELECT COMMUNITY_AREA_NUMBER FROM CrimeData GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1)"
cursor.execute(command_name)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,COMMUNITY_AREA_NAME
0,Austin


9.Use a sub-query to find the name of the community area with highest hardship index

In [98]:
command = "SELECT COMMUNITY_AREA_NAME FROM CensusData WHERE COMMUNITY_AREA_NUMBER = \
(SELECT COMMUNITY_AREA_NUMBER FROM CensusData ORDER BY HARDSHIP_INDEX DESC LIMIT 1);"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,COMMUNITY_AREA_NAME
0,Riverdale


10.Use a sub-query to determine the Community Area Name with most number of crimes?

In [102]:
command = "SELECT COMMUNITY_AREA_NAME FROM CensusData WHERE COMMUNITY_AREA_NUMBER = (\
  SELECT COMMUNITY_AREA_NUMBER FROM CrimeData GROUP BY COMMUNITY_AREA_NUMBEr ORDER BY COUNT(*) DESC LIMIT 1)"
cursor.execute(command)
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
results_df

,COMMUNITY_AREA_NAME
0,Austin
